In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

Choosing variogram model on ordinary kriging...

In [ ]:
scores = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/outputs/grid_searches/ordinary_target_kriging.csv')
scores[scores['rank_test_score'] == 1]

Ordinary spherical model selected

---

Fine tuning on ordinary spherical kriging...

In [ ]:
scores = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/outputs/grid_searches/fine_tuning_target_kriging.csv')
scores[scores['rank_test_score'] == 1].shape

Since the scores are the same, I'll use the model with the lowest fitting time

In [ ]:
scores[scores['mean_fit_time'] == scores['mean_fit_time'].min()].params.iloc[0]

{'nlags': 8, 'variogram_parameters': {'psill': 1.5, 'range': 1.5, 'nugget': -1}, 'weight': False}

In [15]:
from pykrige.rk import Krige
from sklearn.metrics import mean_squared_error

In [26]:
positives = gpd.GeoDataFrame(
    pd.DataFrame(gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_weather_grid.geojson')),
    ).set_crs(epsg=3857, allow_override=True)
negatives = gpd.GeoDataFrame(pd.DataFrame(
    gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_negative_weather_grid.geojson')),
    crs='EPSG:3857')

p = pd.concat([positives[['geometry']], pd.Series([1]*positives.shape[0], name='label')],  axis=1)
n = pd.concat([negatives[['geometry']], pd.Series([0]*negatives.shape[0], name='label')],  axis=1)

tot = pd.concat([p, n])

# Create Kriging model (scikit-learn wrapper)

kriging_model = Krige(method='ordinary', variogram_model='spherical', variogram_parameters={'psill': 1.5, 'range': 1.5, 'nugget': -1}, nlags=8, weight=False)
kriging_model.fit(np.array(pd.concat([tot.geometry.x, tot.geometry.y], axis=1)), tot.label)

In [24]:
ambiguous = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/ambiguous_fires_grid.geojson')

In [31]:
y_pred = kriging_model.predict(np.array(pd.concat([ambiguous.geometry.x, ambiguous.geometry.y], axis=1)))
pred_df = pd.concat([ambiguous, pd.Series(y_pred)], axis=1)

In [37]:
pred_df.rename(columns={0: 'label'}, inplace=True)

In [35]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [38]:
pred_df['DD'] = pred_df.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
pred_df['MM'] = pred_df.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)
pred_df.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_ambiguous_weather_grid.geojson', index=False, driver='GeoJSON')

In [47]:
non_fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_ambiguous_weather_grid.geojson')

In [48]:
from pandas._libs.tslibs.timestamps import Timestamp
non_fires['initialdate'] = non_fires.apply(lambda x: Timestamp(year = x.YYYY, month=x.MM, day=x.DD), axis=1)
non_fires['id'] = [-(idx+100000) for idx in non_fires.index]

In [49]:
non_fires['polygon'] = non_fires.buffer(1000)
fires = non_fires.set_geometry(col='polygon')
fires.drop(columns='geometry', inplace=True)
fires.rename(columns={'polygon': 'geometry'}, inplace=True)
fires.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson',driver='GeoJSON', index=False)

---

In [50]:
f = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson').to_crs('32632')
f.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson',driver='GeoJSON', index=False)

In [ ]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *

In [2]:
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/Nicola_model')
import generate_sentinel_new
reload(generate_sentinel_new)
from generate_sentinel_new import *

In [ ]:
config_path = "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/congif.yaml" # Definisci il percorso del config

config_example = {
        "geojson_path": "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/ambiguous.geojson",
        "interval_pre_fire_days": 7, # AGGIORNATO
        "satellite": "sentinel-2",  # AGGIORNATO
        "patch_size_pixels": 256,
        "target_crs": "EPSG:32632", # NUOVO
        "min_fire_area_sq_m": 200,  # NUOVO
        "image_similarity_tolerance": 0.005 # NUOVO
    }

with open(config_path, "w") as f:
    yaml.dump(config_example, f)
    
print(f"Generato o aggiornato il file di configurazione: {config_path}")


config_data = load_config(config_path) # Carica il config dal percorso specificato

print("\n--- Inizio Processo Batch per TUTTI gli incendi ---")

gdf_all_fires_filtered = gpd.read_file(config_data["geojson_path"])
gdf_all_fires_filtered = gdf_all_fires_filtered[gdf_all_fires_filtered['id'] < -102138]
min_date_for_sentinel2 = datetime(2015, 6, 23) 
    
# # Filtra gli incendi per data (solo quelli dopo l'inizio di Sentinel-2)
# gdf_all_fires_filtered = gdf_all_fires[pd.to_datetime(gdf_all_fires["initialdate"]) >= min_date_for_sentinel2]
    
print(f"Trovati {len(gdf_all_fires_filtered)} incendi nel GeoJSON. Filtrati {len(gdf_all_fires_filtered)} incendi dopo il {min_date_for_sentinel2.strftime('%Y-%m-%d')}.")

for idx, fire_row in gdf_all_fires_filtered.iterrows():
    try:
        process_single_fire(fire_row["id"], config_data, '/nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont')
    except Exception as e:
        print(f"🚨 ERRORE GRAVE durante il processo dell'incendio ID {fire_row['id']}: {e}")
            # Puoi aggiungere qui della logica per registrare gli errori o riprovare

print("\n--- Processo Batch Completato ---")

In [4]:
from PIL import Image

parent = '/nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont'
directories = os.listdir(parent)

black_image = Image.new("RGB", (256, 256), (0, 0, 0))

for fire in directories:
    dir_path = os.path.join(parent, fire)
    if os.path.isdir(dir_path):
        image_path = os.path.join(parent, dir_path, f"{fire}_GTSentinel.png")
        black_image.save(image_path)
        print(f"Saved black image to: {image_path}")
    else:
        print(f"Directory does not exist: {dir_path}")

Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-101207/fire_-101207_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-102271/fire_-102271_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-100645/fire_-100645_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-100113/fire_-100113_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-100657/fire_-100657_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-102242/fire_-102242_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfire-genovese/database/ambiguous_piedmont/fire_-101994/fire_-101994_GTSentinel.png
Saved black image to: /nfs/home/genovese/thesis-wildfir